In [0]:
%sql
create view if not exists dab_project.gold.dim_customers
as 
SELECT
    ci.surrogate_key                   AS customer_key, 
    ci.cst_id                          AS customer_id,
    ci.cst_key                         AS customer_number,
    ci.cst_firstname                   AS first_name,
    ci.cst_lastname                    AS last_name,
    la.cntry                           AS country,
    ci.cst_marital_status              AS marital_status,
    CASE 
        WHEN ci.cst_gender != 'n/a' THEN ci.cst_gender
        ELSE COALESCE(ca.gen, 'n/a')  			   
    END                                AS gender,
    ca.bdate                           AS birthdate,
    ci.cst_create_date                 AS create_date
FROM (select * from delta.`abfss://gold@storagetemp.dfs.core.windows.net/cust_info/`) ci
LEFT JOIN (select * from delta.`abfss://gold@storagetemp.dfs.core.windows.net/cust_az12/`) ca
    ON ci.cst_key = ca.cid
LEFT JOIN (select * from delta.`abfss://gold@storagetemp.dfs.core.windows.net/loc_a101/`)  la
    ON ci.cst_key = la.cid

In [0]:
%sql
create view if not exists dab_project.gold.fact_sales
AS
SELECT
    sd.sls_ord_num  AS order_number,
    pr.product_key  AS product_key,
    cu.customer_key AS customer_key,
    sd.sls_order_dt AS order_date,
    sd.sls_ship_dt  AS shipping_date,
    sd.sls_due_dt   AS due_date,
    sd.sls_sales    AS sales_amount,
    sd.sls_quantity AS quantity,
    sd.sls_price    AS price
FROM (select * from delta.`abfss://gold@storagetemp.dfs.core.windows.net/sales_details/`) sd
LEFT JOIN dab_project.gold.dim_products pr
    ON sd.sls_prd_key = pr.product_number
LEFT JOIN dab_project.gold.dim_customers cu
    ON sd.sls_cust_id = cu.customer_id

In [0]:
%sql

create view if not exists dab_project.gold.dim_products 
as 
SELECT
    pn.surrogate_key AS product_key,
    pn.prd_id       AS product_id,
    pn.prd_key      AS product_number,
    pn.prd_nm       AS product_name,
    pn.cat_id       AS category_id,
    pc.cat          AS category,
    pc.subcat       AS subcategory,
    pc.maintenance  AS maintenance,
    pn.prd_cost     AS cost,
    pn.prd_line     AS product_line,
    pn.prd_start_dt AS start_date
FROM (select * from delta.`abfss://gold@storagetemp.dfs.core.windows.net/prd_info/`) pn
LEFT JOIN (select * from delta.`abfss://gold@storagetemp.dfs.core.windows.net/px_cat_g1v2/`) pc
    ON pn.cat_id = pc.id
WHERE pn.prd_end_dt IS NULL;
